In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import pandas as pd
import holoviews as hv
hv.extension('bokeh')
from holoviews.operation.datashader import datashade

from tqdm import tqdm

import stochastic_sc_model as ssc_model

# Ensure we don't hit floating point errors
np.seterr('raise')

The problem is defined by the location of the currently active polymerases. Each polymerase is characterized by:
1. It's location
2. The length of the nascent mRNA tail ($x$)
3. The DNA twist at this location ($\phi$)
4. The RNAC at this location ($\theta$)

We actually only need to track the first three; the fourth is completely determined by the first three.

Given this state information, we can compute the relaxed supercoiling, which in turn gives us information on the 

In [ ]:
params = {
    'mRNA_drag': 1/20, # pN nm^(alpha / 1)
    'mRNA_exponent': 1, # the value of alpha
    'DNA_twist_mobility': 10, # s pN nm
    'RNAP_radius': 15, # nm
    'RNAP_velocity': 20, # nm / s
    'RNAP_torque_cutoff': 12, # pN nm
    'RNAP_stall_torque_width': 3, #pN
    'DNA_force': 1, # pN
    'DNA_bend_plength': 50, # pN
    'DNA_twist_plength': 95, # pN
    'DNA_plectonome_twist_plength': 24, # pN
    'temperature': 298, # K
    'base_promoter_initiation_rate': 1 / 120, # 1 / sec
    'topo_rate': 1 / 1200, # 1 / sec
    'mRNA_deg_rate': 1 / 1200 # 1 / sec
}

Using a conversion of 1bp = .34nm, we need to investigate our different elements.
The upstream RFP-CYMR ORF is 975bp. WPRE is 589bp, with the polyA signal being 200bp.
The downstream rTTa/IRES/mVenus is 2161bp.

The tandem case is:
upstream-WPRE-polyA-1212bp spacer (UbC)-rTTA combo

The convergent case is:
upstream-WPRE-polyA-589 bp spacer (WPRE)-rTTA combo

Let's make both of the edges a rigid barrier, with that barrier being ~3000bp away on each side (2000 bp viral LTRs, plus 1kb)



In [ ]:
# Create mesh of expression values
mesh_x, mesh_y = np.meshgrid(np.logspace(-1,0,5), np.logspace(-1,0,5))

In [ ]:
RFP_start = 3000 * .34;
RFP_end = RFP_start + (975 + 589 + 200) * .34
mVenus_end = RFP_end + (589 * .34)
mVenus_start = mVenus_end + (2161 * .34)
end_barrier = mVenus_end + (3000 * .34)
convergent = pd.concat([
    ssc_model.bulk_simulation(params, ((0,0), (end_barrier, 0)),[
                            (RFP_start, RFP_end, rfp_strength),
                            (mVenus_start, mVenus_end, mVenus_strength)],
                ['RFP', 'mVenus'], (0, 12000, 1000), 500)
    for rfp_strength, mVenus_strength in tqdm(zip(mesh_x.flatten(), mesh_y.flatten()))])
convergent.to_feather('convergent_promoter_strength_grid.feather')

In [ ]:
convergent = pd.read_feather('convergent_promoter_strength_grid.feather')

In [ ]:
ss_timepoint = convergent[abs((convergent['time'] - 11051)) < 1]

In [ ]:
equal_promoter_strength = ss_timepoint[ss_timepoint['RFP_promoter_strength'] == ss_timepoint['mVenus_promoter_strength']]
equal_promoter_table = hv.Table(equal_promoter_strength,
                                [('RFP_promoter_strength', 'Promoter strength')],
                                ['RFP_expression', 'mVenus_expression'])

In [ ]:
equal_scatter = equal_promoter_table.to.scatter('RFP_expression', 'mVenus_expression')
equal_scatter.overlay('RFP_promoter_strength').opts(
    hv.opts.Scatter(color=hv.Cycle('Category10'), alpha=0.5, size=5),
    hv.opts.NdOverlay(legend_position='top')).options(width=700, height=500)

In [ ]:
equal_scatter.hist(['RFP_expression', 'mVenus_expression'])

In [ ]:
ss_mean = ss_timepoint.groupby(['RFP_promoter_strength', 'mVenus_promoter_strength']).mean()
ss_mean['rfp_venus_ratio'] = ss_mean['RFP_expression'] / ss_mean['mVenus_expression']
ss_mean['rfp_venus_fold_ratio'] = np.log10(ss_mean['rfp_venus_ratio'])

In [ ]:
heatmap = hv.HeatMap(ss_mean, ['mVenus_promoter_strength', 'RFP_promoter_strength'], 'rfp_venus_fold_ratio')
heatmap.opts(colorbar=True, logx=True, logy=True)

In [ ]:
ss_mean.loc[:, ['RFP_expression', 'mVenus_expression', 'rfp_venus_ratio']]

In [ ]:
RFP_start = 3000 * .34;
RFP_end = RFP_start + (975 + 589 + 200) * .34
mVenus_end = RFP_end + (589 * .34)
mVenus_start = mVenus_end + (2161 * .34)
end_barrier = mVenus_end + (3000 * .34)
sim = ssc_model.SupercoilingSimulation(params, ((0,0), (end_barrier, 0)),[
                            (RFP_start, RFP_end, 1),
                            (mVenus_start, mVenus_end, 1)])

sim.enable_topo_relaxation()
single_result = sim.postprocess_run(sim.simulate((0, 12000)))
ssc_model.gen_movie(single_result, [
    (RFP_start, RFP_end, 1),
    (mVenus_start, mVenus_end, 1)], 60 * 60, 'equal', 'animation_output', True)

sim = ssc_model.SupercoilingSimulation(params, ((0,0), (end_barrier, 0)),[
                            (RFP_start, RFP_end, .3),
                            (mVenus_start, mVenus_end, 1)])

sim.enable_topo_relaxation()
single_result = sim.postprocess_run(sim.simulate((0, 12000)))
ssc_model.gen_movie(single_result, [
    (RFP_start, RFP_end, 1),
    (mVenus_start, mVenus_end, 1)], 60 * 60, 'unequal', 'animation_output', True)

In [ ]:
datashade(hv.Points(convergent, ['time', 'RFP_expression'])) * hv.Curve(ss_mean, ['time', 'RFP_expression'])

In [ ]:
plt.plot(np.linspace(0,.05,100),sim.model.polymerase_velocity(np.zeros((100,)), np.linspace(0,.05,100)))
plt.show()
plt.plot(np.linspace(0,20,100), 1 / (1 + np.exp((np.linspace(0,20,100) - 12)/.1)))
plt.show()
plt.plot(np.linspace(0,.1,100), sim.model.torque_response(np.linspace(0,.1,100)))